# Modeling a GraphQL Schema: Pricing Document

**Goal:** Design a schema that exposes a `PricingDocument` with:
- **Catalog** = product/meta info for an ASIN (title, brand, category, description, default currency).
- **Cost** = vendor **procurement** cost for that ASIN (not retail price).

**Architecture intent:**  
- Modular ownership, single API
- Teams own **separate schema files** (e.g., `catalog.graphql`, `cost.graphql`) and their resolvers/data sources.  
- We **merge** these SDLs into **one AppSync API** for consumers → one endpoint, simpler onboarding.  

## GraphQL Type System Fundamentals (5 mins)

**Scalar Types** - Built-in primitives:
- `String`, `Int`, `Float`, `Boolean`, `ID`
- `AWSDateTime` (AppSync-specific)

**Object Types** - Custom data structures:
```
type Product {
 name: String!
 price: Float
}
```

**Input Types** - For mutations and arguments:
```
input ProductInput {
 name: String!
 price: Float!
}
```

**Modifiers**:
- `!` = required (non-null)
- `[String]` = list of strings
- `[String!]!` = required list of required strings

**Arguments** - Parameters for fields:
```
type Query {
 product(id: ID!): Product
 products(limit: Int = 10): [Product]
}
```

## What are "sections" — and why use them?

Sections are **logical groupings** under the document to keep the schema organized and extensible without changing root shape.
- `catalogSection` → fields related to catalog
- `costDataSection` → fields related to cost, right now contains vendor procurement cost, can have other types of cost as well


## Catalog Section

In [1]:
%%writefile schema/schema_asin_pricing.graphql

type CatalogSection {
  catalogData: CatalogData
}

type CatalogData {
  # We omit 'asin' here because parent PricingDocument already provides it
  title: String
  brand: String
  category: String
  description: String
  defaultCurrency: String
  updatedAt: AWSDateTime
}

# ---------- Catalog mutation ----------
input CatalogDataInput {
  title: String
  brand: String
  category: String
  description: String
  defaultCurrency: String
  updatedAt: AWSDateTime
}

Writing schema/schema_asin_pricing.graphql


## 🧩 Practice (3 mins): Add the *vendor cost* section

**Goal:** Define types to read and update the **procurement cost** for a vendor.

**You need to create**
1. A section under the document:  
   `CostDataSection` with one field:  
   `vendorCostData(vendorId: ID!): VendorCost`
2. A payload type:  
   `VendorCost` with fields  
   - `vendorId: ID!`  
   - `cost: Float!`  
   - `currency: String!`  
   - `updatedAt: AWSDateTime`
3. An input type:  
   `VendorCostInput` with the same fields (for writes).

**Hints**
- Don’t add `asin` inside `VendorCost`; the parent document provides it.  
- You’re writing **GraphQL SDL**, not Python code.  
- Keep it minimal — one section, one type, one input


In [2]:
COST_SDL_USER = """
# ---------- COST section (your design) ----------


"""
print("✏️ Got your COST SDL. Run the next cell to preview.")


✏️ Got your COST SDL. Run the next cell to preview.


In [3]:
print(COST_SDL_USER.strip() or "(empty)")

# ---------- COST section (your design) ----------


## 📖 One possible solution for cost types

In [4]:
try:
    with open("schema/00_0_cost_schema.txt") as f:
        COST_SDL = f.read()
    with open("schema/schema_asin_pricing.graphql", "a") as f:
        f.write("\n" + COST_SDL.strip() + "\n")
    print("📖 Our solution:\n")
    print(COST_SDL.strip())
except FileNotFoundError:
    print("⚠️ File 00_0_cost_schema.txt not found. Make sure it exists.")

📖 Our solution:

# ---------- COST section (instructor solution) ----------

type CostDataSection {
  # Retrieves the vendor procurement cost for a given ASIN and vendor.
  vendorCostData(vendorId: ID!): VendorCost
}

# Represents the latest known vendor procurement cost entry.
type VendorCost {
  vendorId: ID!
  cost: Float!
  currency: String!
  updatedAt: AWSDateTime
}

# Input object used for creating or updating vendor cost data.
input VendorCostInput {
  vendorId: ID!
  cost: Float!
  currency: String!
  updatedAt: AWSDateTime
}


## Pricing Document stitching

In [5]:
%%writefile -a schema/schema_asin_pricing.graphql

# ---------- Core document ----------
type PricingDocument {
  asin: ID!
  catalogSection: CatalogSection
  costDataSection: CostDataSection
}

Appending to schema/schema_asin_pricing.graphql


## Query and Mutation Types

In GraphQL, the **root schema** declares *how clients enter* your API:

- `Query`: read-only operations (think: `SELECT`)
- `Mutation`: write/update operations (think: `INSERT/UPDATE`)

In [6]:
%%writefile -a schema/schema_asin_pricing.graphql


type Query {
  # One-stop read for an ASIN
  pricingDocument(asin: String!): PricingDocument!
}

type Mutation {
  putCatalogData(asin: ID!, catalogData: CatalogDataInput): CatalogData
  putVendorCost(asin: ID!, costData: VendorCostInput): VendorCost
}

# ---------- Root entry points ----------
schema {
  query: Query
  mutation: Mutation
}

Appending to schema/schema_asin_pricing.graphql
